In [ ]:
#Dependência usada para o treinamento de modelos de deep learning, manipulação de tensores, GPU (CUDA)
!pip install torch torchvision timm tqdm pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms
from torch.cuda.amp import autocast, GradScaler
from timm import create_model
from PIL import Image
import os
import zipfile
from google.colab import drive   #Importar o módulo do Google Colab drive

# --- Configuração Específica do Colab ---
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado com sucesso.")

# Defina o caminho para o seu conjunto de dados compactado no Google Drive
# IMPORTANTE: Ajuste este caminho se a sua estrutura de pastas for diferente
zipped_dataset_path = '/content/drive/MyDrive/Dataset_librasMono/libras_CNN.zip'

# Defina o caminho de extração dentro do ambiente Colab
extraction_base_path = '/content/libras_CNN_extracted/'

# Crie o diretório de extração se ele não existir
os.makedirs(extraction_base_path, exist_ok=True)

# Descompacte o conjunto de dados
print(f"Descompactando '{zipped_dataset_path}' para '{extraction_base_path}'...")
try:
    with zipfile.ZipFile(zipped_dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_base_path)
    print("Dataset descompactado com sucesso!")
except FileNotFoundError:
    print(f"Erro: O arquivo '{zipped_dataset_path}' não foi encontrado.")
    print("Por favor, verifique o caminho para o seu dataset compactado no Google Drive.")
    exit() # Saia se o conjunto de dados não for encontrado
except Exception as e:
    print(f"Ocorreu um erro durante a descompactação: {e}")
    exit()  # Saia em outros erros de descompactação

# Verifique os arquivos extraídos (Opcional, mas altamente recomendado para depuração)
print("\nConteúdo do dataset descompactado (primeiros itens):")
# Você pode precisar ajustar isso dependendo da estrutura exata dentro do seu zip
# Por exemplo, se libras_CNN.zip contém diretamente as pastas 'train' e 'test':
# list_dir_path = extraction_base_path
# Se libras_CNN.zip contém uma pasta como 'libras_CNN/' que por sua vez contém 'train' e 'test':
list_dir_path = os.path.join(extraction_base_path, 'libras_CNN') # Cenário comum
if os.path.exists(list_dir_path):
    for root, dirs, files in os.walk(list_dir_path):
        level = root.replace(list_dir_path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        if level < 2: # Apenas imprima os primeiros arquivos para evitar uma saída excessiva.
            for f in files[:5]: # Mostre apenas até 5 arquivos por pasta
                print(f'{indent}    {f}')
            if len(files) > 5:
                print(f'{indent}    ... ({len(files) - 5} more files)')
        if level > 2 and not dirs and not files: # Pare mais fundo se não houver conteúdo
            break
else:
    print(f"Aviso: O caminho de verificação '{list_dir_path}' não foi encontrado. Verifique a estrutura do seu zip.")


# --- Adaptação de Código Original ---

def load_data(data_path, img_size, mode='train'):
    # Verificar o modo (treino ou teste) e aplicar as transformações adequadas
    if mode == 'train':
        transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif mode == 'test':
        transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        raise ValueError("O parâmetro 'mode' deve ser 'train' ou 'test'.")

    # Carregar o dataset com o caminho e as transformações apropriadas
    dataset = datasets.ImageFolder(data_path, transform=transform)

    # Retornar o dataset carregado e o número de classes
    num_classes = len(dataset.classes)

    return dataset, num_classes

def split_dataset(dataset, val_ratio=0.2):
    val_size = int(len(dataset) * val_ratio)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset

def define_model(model_name, num_classes):
    model = create_model(model_name, pretrained=True, num_classes=num_classes) # 192
    return model

def train_and_validate(model, train_loader, val_loader, device, epochs=10, lr=5e-5):
    print('\n- - - - - Treinando o modelo - - - - -')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scaler = GradScaler()

    best_val_acc = 0.0  # Melhor acurácia de validação

    for epoch in range(epochs):
        # Treinamento
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # Validação
        model.eval()
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                with autocast():
                    outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / len(val_loader.dataset)
        print(f"Validation Accuracy: {val_acc:.2f}%")

        # Salvar o modelo se for o melhor até agora
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            #print(f"Melhor modelo salvo com acurácia de validação: {best_val_acc:.2f}%")

    #print(f"Melhor Acurácia de Validação: {best_val_acc:.2f}%")

def evaluate_model(model, test_loader, device):
    #print('- - - - - Avaliando o modelo no conjunto de teste - - - - -')
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()

    test_acc = 100 * correct / len(test_loader.dataset)
    print(f"Acurácia no Conjunto de Teste: {test_acc:.2f}%")

    return test_acc

def main():
    IMG_SIZE = (64, 64)
    BATCH_SIZE = 32

    # --- CAMINHOS AJUSTADOS PARA O COLAB ---
    # Assumindo que o arquivo zip seja extraído para uma pasta chamada 'libras_CNN'
    # que então contém subpastas 'treinar' e 'testar'.
    # Ajuste 'libras_CNN' se a pasta raiz extraída tiver um nome diferente.
    # Corrigindo os caminhos para usar o extraction_base_path
    # TRAIN_PATH = os.path.join(extraction_base_path, 'libras_CNN', 'train')
    # TEST_PATH = os.path.join(extraction_base_path, 'libras_CNN', 'test')
    # Se o zip contiver diretamente as pastas 'train' e 'test', use:

    # Caminhos corrigidos com base na saída da listagem do diretório
    TRAIN_PATH = os.path.join(extraction_base_path, 'train')
    TEST_PATH = os.path.join(extraction_base_path, 'test')
    # Você pode precisar verificar a estrutura real de pastas após descompactar

    EPOCHS = 10
    LEARNING_RATE = 0.001

    train_dataset, num_classes = load_data(TRAIN_PATH, IMG_SIZE, 'train')

    test_dataset, _ = load_data(TEST_PATH, IMG_SIZE, 'test')

    train_dataset, val_dataset = split_dataset(train_dataset)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    print('- - - - - - - - - - - - - - - - - - - - -')
    print(f"Caminho do treino: {TRAIN_PATH}")
    print(f"Caminho do teste: {TEST_PATH}")
    print(f"Número de classes: {num_classes}")
    print(f"Quantidade de imagens de treino: {len(train_dataset)}")
    print(f"Quantidade de imagens de validação: {len(val_dataset)}")
    print(f"Quantidade de imagens de teste: {len(test_dataset)}")
    print('- - - - - - - - - - - - - - - - - - - - -')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Dispositivo usado: {device}")

    # Esta linha já está configurada para o modelo 'efficientnet_b3'
    model_names = ['efficientnet_b3']


    accuracies_list1 = []

    for model_name in model_names:
        print(f"\n--- Iniciando treinamento para o modelo: {model_name} ---")
        model = define_model(model_name, num_classes)
        train_and_validate(model, train_loader, val_loader, device, epochs=EPOCHS, lr=LEARNING_RATE)
        print(f"Treino usado: {TRAIN_PATH}")
        print('\n- - - - - - - Avaliando o modelo no conjunto de teste - - - - - - -')
        acc1 = evaluate_model(model, test_loader, device)
        accuracies_list1.append(acc1)

    print(f"\nAcurácias finais dos modelos no conjunto de teste:")
    for model_name, accuracy in zip(model_names, accuracies_list1):
        print(f"{model_name}: {accuracy:.2f}%")

if __name__ == "__main__":
    main()

Montando Google Drive...
Mounted at /content/drive
Google Drive montado com sucesso.
Descompactando '/content/drive/MyDrive/Dataset_librasMono/libras_CNN.zip' para '/content/libras_CNN_extracted/'...
Dataset descompactado com sucesso!

Conteúdo do dataset descompactado (primeiros itens):
Aviso: O caminho de verificação '/content/libras_CNN_extracted/libras_CNN' não foi encontrado. Verifique a estrutura do seu zip.
- - - - - - - - - - - - - - - - - - - - -
Caminho do treino: /content/libras_CNN_extracted/train
Caminho do teste: /content/libras_CNN_extracted/test
Número de classes: 21
Quantidade de imagens de treino: 27772
Quantidade de imagens de validação: 6942
Quantidade de imagens de teste: 11548
- - - - - - - - - - - - - - - - - - - - -
Dispositivo usado: cuda

--- Iniciando treinamento para o modelo: efficientnet_b3 ---


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]


- - - - - Treinando o modelo - - - - -


/tmp/ipython-input-2-2199186293.py:114: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/868 [00:00<?, ?it/s]/tmp/ipython-input-2-2199186293.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10: 100%|██████████| 868/868 [01:44<00:00,  8.30it/s]

...

Epoch 10/10: 100%|██████████| 868/868 [01:41<00:00,  8.55it/s]

Epoch [10/10], Loss: 0.0274


Validation Accuracy: 99.35%
Treino usado: /content/libras_CNN_extracted/train

- - - - - - - Avaliando o modelo no conjunto de teste - - - - - - -


Testing: 100%|██████████| 361/361 [00:12<00:00, 27.97it/s]

Acurácia no Conjunto de Teste: 98.46%

Acurácias finais dos modelos no conjunto de teste:
efficientnet_b3: 98.46%


In [ ]:
import os

extraction_base_path = '/content/libras_CNN_extracted/'

print(f"Conteúdo de '{extraction_base_path}':")
# Listar todos os itens no diretório de extração
for item in os.listdir(extraction_base_path):
    print(item)

# Se houver uma subpasta, você pode querer listar seu conteúdo também.
# Por exemplo, se você suspeitar que há uma pasta chamada 'libras_CNN' dentro dela:
# subfolder_path = os.path.join(extraction_base_path, 'libras_CNN')
# if os.path.exists(subfolder_path):
# print(f"\nConteúdo de '{subfolder_path}':")
# for item in os.listdir(subfolder_path):
# print(item)

Conteúdo de '/content/libras_CNN_extracted/':
train
test


In [ ]:
# Instala as bibliotecas que você vai precisar para a conversão
!pip install onnx onnxruntime onnx-tf tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.4
    Uninstalling typeguard-4.4.4:
      Successfully uninstalled typeguard-4.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


1. Carregar o Modelo .pth e Exportar para ONNX

In [ ]:
import torch
import os
from timm import create_model # Garanta que 'timm' está instalado e disponível

# --- Definir Parâmetros Chave ---
# Esses valores DEVEM ser os mesmos que você usou no TREINAMENTO do seu modelo.
# Se você mudou o IMG_SIZE para 224x224 durante o treinamento, use 224 aqui.
IMG_SIZE = (64, 64) # Adapte para o IMG_SIZE real do seu treinamento
# O número de classes é essencial. Você pode obtê-lo da sua função 'load_data'
# ou simplesmente do número de subpastas (classes) no seu dataset de treinamento.
# Se a sua última execução de 'main()' imprimiu 'Número de classes: X', use esse X.
# Exemplo: NUM_CLASSES = 192
NUM_CLASSES = 192 # <--- ATUALIZE ESTE VALOR COM O NÚMERO REAL DE CLASSES DO SEU DATASET!

MODEL_NAME = 'efficientnet_b3' # O nome do modelo que você treinou

# --- Carregar o Modelo Treinado do .pth ---
print(f"\nCarregando modelo '{MODEL_NAME}' de 'best_model.pth' para exportação...")
# 1. Crie uma nova instância do modelo com a mesma arquitetura usada no treinamento.
# 'pretrained=False' porque você vai carregar seus próprios pesos, não os pré-treinados do ImageNet.
model_to_export = create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES)

# 2. Carregue os pesos que você salvou no arquivo 'best_model.pth'.
# 'map_location=torch.device('cpu')' garante que funcione mesmo se você estiver em CPU ou GPU.
try:
    model_to_export.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))
    print("Pesos do modelo 'best_model.pth' carregados com sucesso!")
except FileNotFoundError:
    print("ERRO: 'best_model.pth' não encontrado. Certifique-se de que seu treinamento foi concluído e o arquivo foi salvo no diretório correto do Colab.")
    print("Você pode verificar clicando no ícone de pasta à esquerda no Colab.")
    exit() # Interrompe a execução se o arquivo não for encontrado
except Exception as e:
    print(f"ERRO ao carregar os pesos do modelo: {e}")
    exit()

# 3. Coloque o modelo em modo de avaliação. Isso desativa camadas como Dropout e Batch Normalization,
# que são usadas apenas durante o treinamento.
model_to_export.eval()

# --- Exportar para ONNX ---
onnx_model_filename = 'model_efficientnet_b3.onnx' # Nome do arquivo ONNX de saída

# 1. Crie um tensor de entrada de exemplo (dummy input).
# Ele simula uma única imagem RGB (3 canais) com as dimensões esperadas pelo seu modelo (IMG_SIZE).
# Formato: (batch_size, channels, height, width)
dummy_input = torch.randn(1, 3, IMG_SIZE[0], IMG_SIZE[1])
print(f"Criado tensor de entrada de exemplo com shape: {dummy_input.shape}")

# 2. Realize a exportação para ONNX.
print(f"Iniciando exportação do modelo para ONNX em: {onnx_model_filename}...")
try:
    torch.onnx.export(model_to_export,
                      dummy_input, # Entrada de exemplo
                      onnx_model_filename, # Nome do arquivo de saída
                      verbose=False, # Defina como True para ver mais detalhes da exportação (útil para debug)
                      input_names=['input'], # Nomeia a entrada do modelo
                      output_names=['output'], # Nomeia a saída do modelo
                      # Permite que o tamanho do batch seja flexível (útil para inferência)
                      dynamic_axes={'input': {0: 'batch_size'},
                                    'output': {0: 'batch_size'}})
    print("Modelo exportado para ONNX com sucesso!")
except Exception as e:
    print(f"ERRO ao exportar para ONNX: {e}")
    print("Verifique se as dependências ONNX estão corretas e se o modelo não tem operações não suportadas.")


Carregando modelo 'efficientnet_b3' de 'best_model.pth' para exportação...
ERRO ao carregar os pesos do modelo: Error(s) in loading state_dict for EfficientNet:
	size mismatch for classifier.weight: copying a param with shape torch.Size([21, 1536]) from checkpoint, the shape in current model is torch.Size([192, 1536]).
	size mismatch for classifier.bias: copying a param with shape torch.Size([21]) from checkpoint, the shape in current model is torch.Size([192]).
Criado tensor de entrada de exemplo com shape: torch.Size([1, 3, 64, 64])
Iniciando exportação do modelo para ONNX em: model_efficientnet_b3.onnx...
Modelo exportado para ONNX com sucesso!


In [ ]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-addons -y
!pip uninstall onnx-tf -y # Pode ser útil reinstalar também

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorflow-addons 0.23.0
Uninstalling tensorflow-addons-0.23.0:
  Successfully uninstalled tensorflow-addons-0.23.0
Found existing installation: onnx-tf 1.10.0
Uninstalling onnx-tf-1.10.0:
  Successfully uninstalled onnx-tf-1.10.0


In [ ]:
# --- CÉLULA 1: Instalação de Dependências e Reinício ---
print("Desinstalando bibliotecas existentes...")
!pip uninstall tensorflow -y
!pip uninstall tensorflow-addons -y
!pip uninstall onnx-tf -y
!pip uninstall tensorflow-probability -y
!pip uninstall keras -y

print("\nInstalando versões compatíveis...")
!pip install tensorflow==2.15.0
!pip install tensorflow-addons==0.23.0
!pip install onnx-tf==1.10.0
!pip install onnx onnxruntime tensorflow-probability==0.23.0

print("\nINSTALAÇÃO CONCLUÍDA. POR FAVOR, REINICIE O AMBIENTE DE EXECUÇÃO (Runtime -> Restart runtime...) AGORA!")
#return # Mantenha este 'return' AQUI para forçar o reinício manual.

Desinstalando bibliotecas existentes...
Found existing installation: tensorflow-probability 0.25.0
Uninstalling tensorflow-probability-0.25.0:
  Successfully uninstalled tensorflow-probability-0.25.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0

Instalando versões compatíveis...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

  Using cached tensorflow_addons-0.23.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached tensorflow_addons-0.23.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (611 kB)
  Using cached onnx_tf-1.10.0-py3-none-any.whl.metadata (510 bytes)
Using cached onnx_tf-1.10.0-py3-none-any.whl (226 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 24.4 MB/s eta 0:00:00

INSTALAÇÃO CONCLUÍDA. POR FAVOR, REINICIE O AMBIENTE DE EXECUÇÃO (Runtime -> Restart runtime...) AGORA!


2: Converter ONNX para TensorFlow Lite (TFLite)

In [ ]:
# --- CÉLULA 2: Conversão ONNX para TFLite (APÓS O REINÍCIO) ---
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf
import os
import shutil # Usado para remover o diretório temporário

# --- Definir Parâmetros Chave (os mesmos que no treinamento e exportação ONNX) ---
# Adapte para o IMG_SIZE real do seu treinamento (ex: (224, 224) se você mudou)
IMG_SIZE = (64, 64)

# --- Definir Caminhos de Arquivos ---
# Nome do arquivo ONNX gerado no Passo 1 (certifique-se que ele existe na raiz do Colab)
onnx_model_filename = 'model_efficientnet_b3.onnx'
# Nome do arquivo TFLite de saída
tflite_model_filename = 'model_efficientnet_b3.tflite'
# Novo diretório temporário
tf_model_temp_dir = 'tf_model_temp_dir_v2'

# --- Verificar se o arquivo ONNX existe ---
if not os.path.exists(onnx_model_filename):
    print(f"ERRO: O arquivo '{onnx_model_filename}' não foi encontrado. Certifique-se de que o Passo 1 foi executado com sucesso e o arquivo foi gerado.")
    exit() # Permanece aqui para parar se o ONNX não existir

print(f"\nCarregando modelo ONNX de: {onnx_model_filename}...")
try:
    onnx_model = onnx.load(onnx_model_filename)
    onnx.checker.check_model(onnx_model)
    print("Modelo ONNX carregado e validado com sucesso.")
except Exception as e:
    print(f"ERRO ao carregar/validar modelo ONNX: {e}")
    exit()

# --- Converter ONNX para TensorFlow SavedModel ---
print("\nConvertendo ONNX para o formato TensorFlow SavedModel...")
try:
    tf_rep = prepare(onnx_model)
    tf_rep.export_graph(tf_model_temp_dir)
    print(f"Modelo TensorFlow SavedModel salvo temporariamente em: {tf_model_temp_dir}")
except Exception as e:
    print(f"ERRO ao converter ONNX para TensorFlow SavedModel: {e}")
    if os.path.exists(tf_model_temp_dir):
        shutil.rmtree(tf_model_temp_dir)
    exit()

# --- Converter TensorFlow SavedModel para TensorFlow Lite ---
print("\nConvertendo TensorFlow SavedModel para TensorFlow Lite (.tflite)...")
try:
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_temp_dir)

    # Otimizações Opcionais (recomendado)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    tflite_model = converter.convert()

    with open(tflite_model_filename, 'wb') as f:
        f.write(tflite_model)

    print(f"Modelo TFLite salvo com sucesso em: {tflite_model_filename}")

except Exception as e:
    print(f"ERRO ao converter TensorFlow SavedModel para TFLite: {e}")
finally:
    if os.path.exists(tf_model_temp_dir):
        shutil.rmtree(tf_model_temp_dir)
        print(f"Diretório temporário '{tf_model_temp_dir}' removido.")

# --- Baixar o Modelo TFLite ---
from google.colab import files
try:
    files.download(tflite_model_filename)
    print(f"\nArquivo '{tflite_model_filename}' baixado com sucesso!")
except Exception as e:
    print(f"Erro ao baixar o arquivo: {e}")
    print(f"Verifique se '{tflite_model_filename}' existe no diretório.")

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(



Carregando modelo ONNX de: model_efficientnet_b3.onnx...
Modelo ONNX carregado e validado com sucesso.

Convertendo ONNX para o formato TensorFlow SavedModel...
Modelo TensorFlow SavedModel salvo temporariamente em: tf_model_temp_dir_v2

Convertendo TensorFlow SavedModel para TensorFlow Lite (.tflite)...
Modelo TFLite salvo com sucesso em: model_efficientnet_b3.tflite
Diretório temporário 'tf_model_temp_dir_v2' removido.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Arquivo 'model_efficientnet_b3.tflite' baixado com sucesso!
